In [1]:
import pandas as pd
import numpy as np
from numpy import random as r
from matplotlib import pyplot as plt

In [4]:
import Quandl as q
path="C:\\dev\\psignal\\data\\"

In [3]:
#create a sample
pureTwt='twitter_noretweets_daily.csv'
ptwt=pd.read_csv(filepath_or_buffer=path+pureTwt,delimiter=",",header=0)
ptwt=ptwt.sample(n=1000)
ptwt['TIMESTAMP_UTC']=pd.to_datetime(arg=ptwt.TIMESTAMP_UTC,infer_datetime_format='true',unit='D')
#ptwt['TIMESTAMP_UTC']=pd.to_datetime(arg=ptwt.TIMESTAMP_UTC,format='%Y%m%d')
cols=['SYMBOL','TIMESTAMP_UTC']
ptwt=ptwt.reindex(columns=cols)
ptwt.columns=['sym','date']
ptwt['cluster']=r.randint(low=1,high=5,size=len(ptwt))
ptwt.to_csv("sample ptwt.csv")
#this is how data should look like after clustring
ptwt.head()

,sym,date,cluster
354265,ARMH,2014-06-26 04:00:00,2
4529212,SODA,2015-03-29 04:00:00,1
66103,ACO,2014-03-12 04:00:00,1
433856,AUDUSD,2013-03-04 05:00:00,4
803372,CARA,2014-06-20 04:00:00,1


In [7]:
#if sample already exists use this to import it
sampleTWT='sample ptwt.csv'
ptwt=pd.read_csv(filepath_or_buffer=path+sampleTWT,delimiter=",",header=0)
ptwt['date']=pd.to_datetime(arg=ptwt.date,infer_datetime_format='true',unit='D')

In [8]:
ptwt.head()

,Unnamed: 0,sym,date,cluster
0,3036178,LULU,2011-04-06 04:00:00,1
1,1081119,CONGLOMERATES,2011-12-14 05:00:00,2
2,204689,ALKS,2012-08-11 04:00:00,2
3,1105968,CP.CA,2014-09-23 04:00:00,2
4,3603262,NZ,2011-02-23 05:00:00,2


In [9]:
def toYMD(x):
    return x.date()
ptwt['date']=ptwt['date'].map(toYMD)

In [10]:
ptwt.head()

,Unnamed: 0,sym,date,cluster
0,3036178,LULU,2011-04-06,1
1,1081119,CONGLOMERATES,2011-12-14,2
2,204689,ALKS,2012-08-11,2
3,1105968,CP.CA,2014-09-23,2
4,3603262,NZ,2011-02-23,2


In [41]:
#data is imported from clustering stage
#data = pd.read_csv('../data/data2010to2015.csv')
#format is: date | sym  | cluster
#date is daily
#sym is cashtag/ticker
#cluster is a number that indicates cluster
data=ptwt
tickers = ['LULU','CONGLOMERATES','ALKS'] #data['sym'].unique()
#rdata=pd.DataFrame({})
for sym in tickers:   
    try:
        #can later expand this to include search from other db than WIKI
        tmp=q.get(dataset=("WIKI/"+sym+".11"),trim_start='2010-01-01', trim_end='2016-01-01', transformation='rdiff',\
                  authtoken="c54mBskiz_BsF4vWWL2s")
        tmp['sym']=sym
        tmp['date']=tmp.index
        tmp['date']=tmp['date'].map(toYMD)
    except:
        tmp= pd.DataFrame({'sym':[],'date':[]})
        print("Dataset "+sym+" not found!")
    data=pd.merge(data,tmp,on=['sym','date'],how='left') #assumes date in data is indeed daily

print(data.head())

mi = q.get("YAHOO/INDEX_GSPC.6",start_date='2010-01-01', end_date='2016-01-01', transformation='rdiff',\
                  authtoken="c54mBskiz_BsF4vWWL2s")

Dataset LULU not found!
Dataset CONGLOMERATES not found!
   Unnamed: 0            sym       date  cluster  Adj. Close
0     3036178           LULU 2011-04-06        1         NaN
1     1081119  CONGLOMERATES 2011-12-14        2         NaN
2      204689           ALKS 2012-08-11        2         NaN
3     1105968          CP.CA 2014-09-23        2         NaN
4     3603262             NZ 2011-02-23        2         NaN


In [55]:
a=pd.date_range(start='2010-01-01',end='2015-01-01',freq='1M')
ts=pd.DataFrame({'r': np.randn(len(a),1)})
ts.set_index(a)
ts.resample('2M')

AttributeError: 'module' object has no attribute 'randn'

In [49]:
tmp['date'][tmp['date']=='2012-08-20']

Series([], Name: date, dtype: object)

In [35]:
data.dtypes

Unnamed: 0        int64
sym              object
date             object
cluster           int64
Adj. Close_x    float64
Adj. Close_y    float64
Adj. Close      float64
dtype: object

In [28]:
sym='AAPL'
tmp=q.get(dataset=("WIKI/"+sym+'.11'),trim_start='2010-01-01', trim_end='2016-01-01', transformation='rdiff',\
                  authtoken="c54mBskiz_BsF4vWWL2s")
tmp.head()

,Adj. Close
Date,
2010-01-05,0.001729
2010-01-06,-0.015906
2010-01-07,-0.001849
2010-01-08,0.006648
2010-01-11,-0.008822


In [14]:
data.head()

,Unnamed: 0,sym,date,cluster
0,3036178,LULU,2011-04-06,1
1,1081119,CONGLOMERATES,2011-12-14,2
2,204689,ALKS,2012-08-11,2
3,1105968,CP.CA,2014-09-23,2
4,3603262,NZ,2011-02-23,2


In [ ]:
def cumret(ts):
    ts=np.add(ts,1)
    ts=np.cumprod(ts)[len(ts)-1]
    ts = ts-1
    return ts

data = data.resample(bmth_us, how=cumret)
mi = mi.resample(bmth_us, how=cumret)

In [120]:
windows=pd.period_range(start='2010',end='2016',freq="M")
windows=windows.format()
windows[0]

'2010-01'

In [ ]:
import <regression> as reg
for stock in ptwt['sym'].unique():
    for window_end_idx in range(1,len(windows)):
        mask=(ptwt['sym']==stock)&(ptwt['date']>=windows[window_end_idx-1])&(ptwt['date']<windows[window_end_idx])
        tmp=ptwp[mask]
        r_t_plus_one=tmp['return'].ix[1:]
        r_t=tmp['return'].ix[0:len(tmp)-1]
        mask_market_index=(ptwt['date']>=windows[window_end_idx-1])&(ptwt['date']<windows[window_end_idx])
        market_index=SP500[mask].ix[0:len(tmp)-1] #check and correct for stationarity
        SIR=ptwt[mask_market_index].groupby('cluster','date').mean() #compute mean return for each day and cluster
        stationarity_test=reg.adftest(r_t_plus_one, kind="c")
        while (stationarity_test):
            r_t_plus_one=reg.takeFirstDifference(r_t_plus_one)
            r_t=reg.takeFirstDifference(r_t)
            stationarity_test=reg.adftest(r_t_plus_one, kind="c")
        tmp_reg=reg.GLS(y=r_t_plus_one,x=[r_t,market_index],include_constant=True,return_residuals=True)
        tmp_reg_prime=reg.GLS(y=tmp_reg.residuals,x=[r_t,SIR],include_constant=True,return_residuals=True)
        regression[(stock, window)]=(tmp_reg,tmp_reg_prime)

In [122]:
mask=(ptwt['sym']=='ALE')&(ptwt['date']>='2015-1')&(ptwt['date']<='2015-12')
ptwt[mask]

,sym,date,return
3,ALE,2015-01-16 05:00:00,1.54638
